<a href="https://colab.research.google.com/github/aditya-shriwastava/colab-notebooks/blob/master/visual_odometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO
- [ ] Compute trajectory in 3D (Up to scale)
- [ ] Compare trajectory with ground truth data. (Whthout bundle adjustment)
- [ ] Pose-Graph Optimization

# pip3 install

In [ ]:
!pip3 install scikit-image

# import

In [ ]:
import os

import numpy as np

import cv2
from google.colab.patches import cv2_imshow

import matplotlib.pyplot as plt
%matplotlib inline

from skimage.measure import ransac

from tqdm import tqdm, trange

# Load kitti dataset

In [ ]:
if "2011_09_26_drive_0020_sync.zip" not in os.listdir():
  !wget https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0020/2011_09_26_drive_0020_sync.zip
  !unzip  2011_09_26_drive_0020_sync.zip >/dev/null

dataset_dir = "./2011_09_26/2011_09_26_drive_0020_sync/image_02"
imgs_dir = os.path.join(dataset_dir, "data")
timestamps_file_path = os.path.join(dataset_dir, "timestamps.txt")

imgs_file_name = os.listdir(imgs_dir)
imgs_file_name.sort()

imgs_file_path = [os.path.join(imgs_dir, img_file_name) for img_file_name in imgs_file_name]
kitti_imgs = np.array([cv2.imread(img_file_path) for img_file_path in imgs_file_path])

In [ ]:
kitti_cam_calib = np.array([[7.215377e+02,0.000000e+00,6.095593e+02],
                            [0.000000e+00,7.215377e+02,1.728540e+02],
                            [0.000000e+00,0.000000e+00,1.000000e+00]])

# Load deer_robot dataset

In [ ]:
if "deer_robot.zip" not in os.listdir():
  !wget https://www.doc.ic.ac.uk/~wl208/lmdata/deer_robot.zip
  !unzip  deer_robot.zip >/dev/null

imgs_dir = "./deer_robot/cam0/data"
metadata_file = "./deer_robot/cam0/data.csv"
poses_gt_file = "./deer_robot/poses.gt"

def load_metadata():
  with open(metadata_file, 'r') as fd:
      metadata = fd.readlines()

  imgs_file = [
      line.split(',')[1][:-1] for line in metadata[1:]
  ]
  imgs_path = [os.path.join(imgs_dir, f) for f in imgs_file]
  timestamps = [float(line.split(',')[0])/1e9 for line in metadata[1:]]
  return imgs_path, timestamps

def load_poses_gt():
  with open(poses_gt_file, 'r') as fd:
    positions = []
    orientations = []
    for line in fd:
      if line[0] == '#':
        continue

      split = line.split(',')
      px = float(split[1])
      py = float(split[2])
      pz = float(split[3])
      p = [px, py, pz]
      positions.append(p)

      qw = float(split[4])
      qx = float(split[5])
      qy = float(split[6])
      qz = float(split[7])
      q = [qx, qy, qz, qw]
      orientations.append(q)
  return np.array(positions), np.array(orientations)

def load_batch(start, size):
  imgs_path, timestamps = load_metadata()
  imgs = np.array([
      cv2.imread(img_path) for img_path in imgs_path[start:start+size]
  ])
  p, q = load_poses_gt()
  return imgs, timestamps[start:start+size], p[start:start+size], q[start:start+size]

deer_robot_imgs, deer_robot_timestamps, deer_robot_p, deer_robot_q = load_batch(0, 100)

In [ ]:
deer_robot_cam_calib = np.array([
  [600, 0, 320],
  [0, 600, 240],
  [0,   0,   1]
], dtype=np.float)

# Disp utils

In [ ]:
def disp(img_tn1, kps_tn1, kps_t):
  disp_img = img_tn1.copy()
  for kp_tn1, kp_t in zip(kps_tn1, kps_t):
    start_point = tuple(kp_tn1)
    end_point = tuple(kp_t)
    disp_img = cv2.circle(disp_img, start_point, 1, (0, 0, 255), 2)
    disp_img = cv2.line(disp_img, start_point, end_point, (0, 255, 0), 1)
  cv2_imshow(disp_img)

def disp2(img_tn2, kps_tn2, kps_tn1, kps_t):
  disp_img = img_tn2.copy()
  for kp_tn2, kp_tn1, kp_t in zip(kps_tn2, kps_tn1, kps_t):
    start_point = tuple(kp_tn2)
    middle_point = tuple(kp_tn1)
    end_point = tuple(kp_t)
    disp_img = cv2.circle(disp_img, start_point, 1, (0, 0, 255), 2)
    disp_img = cv2.line(disp_img, start_point, middle_point, (0, 255, 0), 1)
    disp_img = cv2.line(disp_img, middle_point, end_point, (255, 0, 0), 1)
  cv2_imshow(disp_img)

# Essential matrix utils

In [ ]:
def triangulate(x_tn1, x_t, R, t):
  """
  Args:
    x_tn1: numpy array of shape (3,) representing normalized homogeneous
      coordinate in img_tn1 of point to triangulate.
    x_t: numpy array of shape (3,) representing normalized homogeneous
      coordinate in img_t of point to triangulate.
    R: numpy array of shape (3,3) representing rotation of camera frame
      at time t wrt camera frame at time tn1.
    t: numpy array of shape (3,) representing baseline vector between 
    camera frame at time t and camera frame at time tn1.
  """
  A = np.array([
    [np.dot(x_tn1, x_tn1), -np.dot(R @ x_t, x_tn1)],
    [np.dot(x_tn1, R @ x_t), -np.dot(R @ x_t, R @ x_t)]
  ])
  b = np.array([np.dot(t, x_tn1) ,np.dot(t, R @ x_t)])
  alpha, beta = np.linalg.inv(A) @ b
  p_tn1 = alpha * x_tn1
  p_t = t + (beta * (R @ x_t))
  return (p_tn1 + p_t)/2

def essential_to_Rt(E, x_tn1, x_t):
  """
  Args:
    E: numpy array of shape (3,3) representing Essential matrix.
    x_tn1: numpy array of shape (3,) representing normalized homogeneous
      coordinate in img_tn1.
    x_tn1: numpy array of shape (3,) representing normalized homogeneous
      coordinate in img_t corresponding to x_tn1.
  """
  u,s,vt = np.linalg.svd(E)
  w = np.array([[0,-1,0],
                [1,0,0],
                [0,0,1]])
  R1 = u @ w.T @ vt
  if np.linalg.det(R1) < 0:
    R1 = -R1
  R2 = u @ w @ vt
  if np.linalg.det(R2) < 0:
    R2 = -R2

  t1 = u @ w @ np.diag(s) @ u.T
  t1 = np.array([t1[2,1], t1[0,2], t1[1,0]])
  t1 = t1/np.linalg.norm(t1)
  t2 = -t1

  # Resolve ambiguities by triangulation
  p11_tn1 = triangulate(x_tn1, x_t, R1, t1)
  p11_t = R1.T @ (p11_tn1 - t1)
  if p11_tn1[2] > 0 and p11_t[2] > 0:
    return R1, t1

  p12_tn1 = triangulate(x_tn1, x_t, R1, t2)
  p12_t = R1.T @ (p12_tn1 - t2)
  if p12_tn1[2] > 0 and p12_t[2] > 0:
    return R1, t2
  
  p21_tn1 = triangulate(x_tn1, x_t, R2, t1)
  p21_t = R2.T @ (p21_tn1 - t1)
  if p21_tn1[2] > 0 and p21_t[2] > 0:
    return R2, t1

  p22_tn1 = triangulate(x_tn1, x_t, R2, t2)
  p22_t = R2.T @ (p22_tn1 - t2)
  if p22_tn1[2] > 0 and p22_t[2] > 0:
    return R2, t2
  
  raise Exception('Failed to resolve ambiguities!!')

In [ ]:
class EssentialMatrixTransform():
  def __init__(self):
    # param contains essential matrix
    self.params = np.eye(3)

  def estimate(self, kps_tn1, kps_t):
    """
    Args:
      kps_tn1: numpy array of shape (N,3) representing Keypoints in
        normalized homogeneous coordinate from img_tn1
      kps_t: numpy array of shape (N,3) representing Keypoints in
        normalized homogeneous coordinate from img_t
    """
    # Setup constraint matrix
    A = np.column_stack((
      kps_tn1[:,0] * kps_t[:,0],
      kps_tn1[:,0] * kps_t[:,1],
      kps_tn1[:,0] * kps_t[:,2],
      kps_tn1[:,1] * kps_t[:,0],
      kps_tn1[:,1] * kps_t[:,1],
      kps_tn1[:,1] * kps_t[:,2],
      kps_tn1[:,2] * kps_t[:,0],
      kps_tn1[:,2] * kps_t[:,1],
      kps_tn1[:,2] * kps_t[:,2]
    ))

    # Solve for nullsapce of the constraint matrix
    u, s, vt = np.linalg.svd(A)
    E = vt[-1,:].reshape(3,3)

    # Enforcing Constraints:
    # 1. First and Second singular values should be equal
    # 2. Third singular value should be zero
    u, s, vt = np.linalg.svd(E)
    s[0] = s[1] = (s[0] + s[1])/2
    s[2] = 0
    self.params = u @ np.diag(s) @ vt

    return True

  def residuals(self, kps_tn1, kps_t):
    """
    Args:
      kps_tn1: numpy array of shape (N,3) representing Keypoints in
        normalized homogeneous coordinate from img_tn1
      kps_t: numpy array of shape (N,3) representing Keypoints in
        normalized homogeneous coordinate from img_t
    """
    e_tn1 = kps_t @ self.params.T  # Epipilar line on tn1 image
    e_t = kps_tn1 @ self.params  # Epipilar line on t image
    
    residue = (kps_tn1 * e_tn1).sum(axis = 1) # Residue from coplanarity constraint

    return np.abs(residue) / np.sqrt(e_tn1[:,0]**2 + e_tn1[:,1]**2
                                     + e_t[:,0]**2 + e_t[:,1]**2)

## Test

In [ ]:
img_tn1 = kitti_imgs[0]
img_t = kitti_imgs[1]

# Ground truth corresponding Keypoints in img_tn1 and img_t
kps_tn1 = np.array([[838,233],[508,73],[514,180],[468,212],[587,141],[492,265],[986,212],[633, 42],[872,311],[544,124],
                    [804,292],[858,247],[689,264],[161,256],[574,184],[637,80],[532,200],[514,202],[564,152],[586,169]])

kps_t = np.array([[ 848,235],[506,72],[512,180],[462,212],[587,141],[486,268],[1037,217],[632,39],[886,317],[542,124],
                  [814,298],[867,249],[690,267],[131,262],[572,184],[636,79],[528,200],[512,202],[564,152],[585,169]])
kps_tn1_ = np.column_stack((kps_tn1, np.ones(len(kps_tn1)))) @ np.linalg.inv(kitti_cam_calib).T
kps_t_ = np.column_stack((kps_t, np.ones(len(kps_t)))) @ np.linalg.inv(kitti_cam_calib).T

disp(img_tn1, kps_tn1, kps_t)

E = EssentialMatrixTransform()
E.estimate(kps_tn1_, kps_t_)

print(f"Essential Matrix: {E.params}")

print("Residuals for corresponding points:")
print(E.residuals(kps_tn1_, kps_t_))

print("Residuals for random points:")
permutation = np.random.choice(range(20), replace=False, size=20)
print(E.residuals(kps_tn1_, kps_t_[permutation]))

# Features utils

In [ ]:
def extract_features(img):
  """
  Args
    img: numpy array of shape (H,W,C) representing image
  Returns
    kps: numpy array of shape (N, 2) representing coordinates of N
      keypoints detected
    des: numpy array of shape (N, 32) representing 32 dim descriptor
      of N keypoints
  """
  img = np.expand_dims(np.mean(img, axis=2).astype(np.uint8), -1)
  orb = cv2.ORB_create(nfeatures=5000)
  kps, des = orb.detectAndCompute(img, None)
  kps = np.array([[kp.pt[0], kp.pt[1]] for kp in kps]).astype(np.int32)
  return kps, des


def match_features(des_tn1, des_t):
  bf_matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
  matches = bf_matcher.match(des_tn1, des_t)
  return np.array([[match.queryIdx, match.trainIdx] for match in matches])

def match_features2(des_tn1, des_t):
  bf_matcher = cv2.BFMatcher(cv2.NORM_HAMMING)
  matches = bf_matcher.knnMatch(des_tn1, des_t, k=2)

  idx_tn1, idx_t = [], []
  for m1, m2 in matches:
    # Lowe's ratio test
    if m1.distance <= 1 * m2.distance:
      if m1.distance < 32:
        if m1.trainIdx not in idx_t:
          idx_tn1.append(m1.queryIdx)
          idx_t.append(m1.trainIdx)

  assert len(idx_tn1) >= 8, f"#matches: {len(idx_tn1)}" 
  return np.array([idx_tn1, idx_t]).T

# Frame

In [ ]:
class Frame:
  def __init__(self, img, K):
    self._img = img
    self._K = K
    self._kps, self._des = extract_features(img)
    self._kps_ = self.normalize(self._kps)

    self.frame2 = None
    self.matches2 = None
    self.E = None
    self.R = None
    self.t = None

    self.matches3 = None
    self.relative_scale = None

  def triangulate(self, kps_idx):
    assert self.frame2 is not None
    matches2_idx, = np.where(self.matches2[:,0] == np.array(kps_idx))
    assert len(matches2_idx) == 1
    matches2_idx = matches2_idx[0]

    return triangulate(
      self.kps_[kps_idx],
      self.frame2.kps_[self.matches2[matches2_idx,1]],
      self.R,
      self.t
    )

  def match2(self, frame2):
    self.frame2 = frame2
    self.matches2 = match_features2(self.des, self.frame2.des)

    self.E, inliers = ransac(
      (self.kps_[self.matches2[:,0]], self.frame2.kps_[self.matches2[:,1]]),
      EssentialMatrixTransform,
      min_samples=8,
      residual_threshold=1e-3,
      max_trials=100
    )
    self.matches2 = self.matches2[inliers]

    self.R, self.t = essential_to_Rt(
      self.E.params,
      self.kps_[self.matches2[0,0]],
      self.frame2.kps_[self.matches2[0,1]]
    )
  
  def match3(self):
    assert self.frame2 is not None
    assert self.frame2.frame2 is not None
    
    self.matches3 = []
    for i, kp in enumerate(self.matches2[:,1]):
      j, = np.where(self.frame2.matches2[:,0] == np.array(kp))
      assert len(j) in [0,1]
      if len(j) == 1:
        j = j[0]
        assert self.matches2[i,1] == self.frame2.matches2[j,0]
        self.matches3.append([self.matches2[i,0], self.matches2[i,1], self.frame2.matches2[j,1]])
    self.matches3 = np.array(self.matches3)
    self.update_relative_scale()

  def update_relative_scale(self):
    indices = np.random.choice(np.arange(len(self.matches3)), 10, replace=False).reshape(5,2)

    relative_scale = []
    for i, j in indices:
      x_i_12 =self.triangulate(self.matches3[i,0])
      x_j_12 =self.triangulate(self.matches3[j,0])
      x_i_23 =self.frame2.triangulate(self.matches3[i,1])
      x_j_23 =self.frame2.triangulate(self.matches3[j,1])

      relative_scale.append(
        np.linalg.norm(x_i_12 - x_j_12)/
        np.linalg.norm(x_i_23 - x_j_23)
      )
    self.relative_scale = np.median(relative_scale)

  def normalize(self, kps):
    return np.column_stack((
      kps,
      np.ones(len(kps))
    )) @ np.linalg.inv(self._K).T

  @property
  def img(self):
    return self._img

  @property
  def kps(self):
    return self._kps

  @property
  def kps_(self):
    return self._kps_

  @property
  def des(self):
    return self._des

# Main

In [ ]:
# idxs = np.arange(100).reshape(50,2)[:,0]
idxs = np.arange(len(kitti_imgs))

frames = []
for idx in tqdm(idxs):
  frames.append(Frame(kitti_imgs[idx], kitti_cam_calib))
  if len(frames) >= 2:
    frames[-2].match2(frames[-1])
    if len(frames) >= 3:
      frames[-3].match3()

In [ ]:
i = 0
disp2(
  frames[i].img,
  frames[i].kps[frames[i].matches3[:,0]],
  frames[i + 1].kps[frames[i].matches3[:,1]],
  frames[i + 2].kps[frames[i].matches3[:,2]]
)

In [ ]:
print(deer_robot_p.shape)
plt.plot(deer_robot_p[:,1], deer_robot_p[:,0])
plt.show()